<a href="https://colab.research.google.com/github/TonyQ7/BE603-Project/blob/main/BE603_Project_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
# CELL 1: SETUP & IMPORTS
# !pip install pandas numpy statsmodels scipy matplotlib seaborn openpyxl linearmodels

import pandas as pd
import numpy as np

# comment out the next 2 lines in Colab
import matplotlib
matplotlib.use('Agg')

import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.stats.api as sms
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 200)
pd.set_option('display.float_format', '{:.4f}'.format)
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

print("Libraries loaded")

Libraries loaded


In [30]:
# ==============================================================================
# CELL 2: AUTOMATED DATA LOADING FROM GITHUB
# ==============================================================================
# This block replaces the manual file upload. It fetches directly from GitHub.

import pandas as pd
# io and requests are handled internally by pandas for these URLs

# 1. Define the Raw GitHub URLs
# Configured for Repo: tonyq7/be603-project | Branch: main

# These links point to the .xlsx files.
# NOTE: We use the "raw" link format to ensure we get the file content, not the GitHub web page.
URL_CAMPAIGNS = 'https://github.com/TonyQ7/BE603-Project/raw/refs/heads/main/BE603_campaigns_09.xlsx'
URL_CONTRIB = 'https://github.com/TonyQ7/BE603-Project/raw/refs/heads/main/BE603_contrib_09.xlsx'

print("Downloading data from GitHub (Branch: main)...")

try:
    # Load Campaign Data
    # CHANGED: Use read_excel() because the source files are .xlsx
    df_camp = pd.read_excel(URL_CAMPAIGNS)

    # Load Contribution Data
    # CHANGED: Use read_excel() because the source files are .xlsx
    df_cont = pd.read_excel(URL_CONTRIB)

    print("=" * 70)
    print("DATA LOADING SUMMARY")
    print("=" * 70)
    print(f"[DATA] Campaign Data: {df_camp.shape[0]:,} rows loaded from GitHub")
    print(f"[DATA] Contribution Data: {df_cont.shape[0]:,} rows loaded from GitHub")

except Exception as e:
    print(f"[ERROR] Could not load data from GitHub.")
    print(f"Details: {e}")
    print("\nTroubleshooting:")
    print("1. Confirm the repository 'tonyq7/be603-project' is still Public.")
    print("2. If seeing 'Engine' errors, try installing openpyxl: !pip install openpyxl")

DATA LOADING SUMMARY
[DATA] Campaign Data: 5,000 rows loaded from GitHub
[DATA] Contribution Data: 8,000 rows loaded from GitHub


In [3]:
# CELL 3: DATA VALIDATION - CAMPAIGN DATASET
print("\n" + "="*70)
print("DATA VALIDATION: CAMPAIGN DATASET")
print("="*70)

# missing values
print("\n1. Missing Values:")
missing = df_camp.isnull().sum()
if missing.sum() == 0:
    print("   No missing values")
else:
    print(missing[missing > 0])

# data types
print("\n2. Data Types:")
print(df_camp.dtypes)

# duplicates
print(f"\n3. Duplicate Rows: {df_camp.duplicated().sum()}")

# quick summary
print("\n4. Key Variables Summary:")
key_vars = ['collected_funds', 'goal', 'campaign_quality', 'comments_count',
            'updates_count', 'reach30in2']
print(df_camp[key_vars].describe().T[['count', 'mean', 'std', 'min', 'max']])

# reach30in2 is binary (True if 30% of goal reached in first 2 days)
print("\n5. reach30in2 Distribution:")
print(df_camp['reach30in2'].value_counts(normalize=True).round(4))


DATA VALIDATION: CAMPAIGN DATASET

1. Missing Values:
   No missing values

2. Data Types:
collected_funds       int64
campaign_quality      int64
goal                  int64
images                int64
video                 int64
comments_count        int64
updates_count         int64
creators              int64
business_venture      int64
social                int64
pitch_size            int64
filler_words        float64
duration            float64
reach30in2             bool
posemo              float64
negemo              float64
time                  int64
dtype: object

3. Duplicate Rows: 0

4. Key Variables Summary:
                     count      mean       std       min        max
collected_funds  5000.0000 2950.0430 3914.9969  500.0000 84820.0000
goal             5000.0000 6214.9004 5021.0111 1000.0000 24800.0000
campaign_quality 5000.0000    5.8760    2.6692    0.0000     9.0000
comments_count   5000.0000   16.7560   26.3605    0.0000   745.0000
updates_count    5000.0000   

In [4]:
# CELL 4: DATA VALIDATION - CONTRIBUTION DATASET
print("\n" + "="*70)
print("DATA VALIDATION: CONTRIBUTION DATASET")
print("="*70)

grouped = df_cont.groupby('id')
print(f"\n1. Panel Structure:")
print(f"   Campaigns: {df_cont['id'].nunique()}")
print(f"   Days per campaign: {df_cont.groupby('id').size().unique()}")

# check time-invariant vars
time_invariant = ['goal', 'collected_funds', 'comments_count', 'updates_count',
                  'creators', 'pitch_size', 'focuspast', 'focuspresent',
                  'focusfuture', 'posemo', 'negemo']

print("\n2. Time-Invariant Variable Check:")
all_ok = True
for col in time_invariant:
    if col in df_cont.columns:
        nunique = grouped[col].nunique()
        bad = nunique[nunique > 1]
        if len(bad) > 0:
            print(f"   WARNING: '{col}' varies within {len(bad)} campaigns")
            all_ok = False
if all_ok:
    print("   All time-invariant variables are constant within campaigns")

# check cumulative perk
print("\n3. daily_total_perk Check:")
df_sorted = df_cont.sort_values(['id', 'day']).copy()
df_sorted['lag_total'] = df_sorted.groupby('id')['daily_total_perk'].shift(1).fillna(0)
df_sorted['diff'] = df_sorted['daily_total_perk'] - df_sorted['lag_total'] - df_sorted['daily_perk']
max_diff = df_sorted['diff'].abs().max()
print(f"   Max inconsistency: {max_diff:.6f}")


DATA VALIDATION: CONTRIBUTION DATASET

1. Panel Structure:
   Campaigns: 200
   Days per campaign: [40]

2. Time-Invariant Variable Check:
   All time-invariant variables are constant within campaigns

3. daily_total_perk Check:
   Max inconsistency: 0.000000


In [5]:
# CELL 5: DESCRIPTIVE STATISTICS
print("\n" + "="*70)
print("DESCRIPTIVE STATISTICS")
print("="*70)

stats_df = df_camp.describe().T
stats_df['skewness'] = df_camp.skew()

# extra/supplementary - kurtosis not required by course
stats_df['kurtosis'] = df_camp.kurtosis()

print("\n" + stats_df[['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max', 'skewness']].to_string())

print("\nHighly Skewed Variables (|skew| > 1):")
high_skew = stats_df[stats_df['skewness'].abs() > 1][['mean', 'skewness']]
print(high_skew)

# funding ratio
df_camp['funding_ratio'] = df_camp['collected_funds'] / df_camp['goal']
print(f"\nFunding Ratio Stats:")
print(df_camp['funding_ratio'].describe())
print(f"Campaigns that reached goal: {(df_camp['funding_ratio'] >= 1).sum()} ({(df_camp['funding_ratio'] >= 1).mean()*100:.1f}%)")


DESCRIPTIVE STATISTICS

                     count      mean       std       min       25%       50%       75%        max  skewness
collected_funds  5000.0000 2950.0430 3914.9969  500.0000 1000.0000 1712.5000 3377.0000 84820.0000    6.7066
campaign_quality 5000.0000    5.8760    2.6692    0.0000    4.0000    6.0000    9.0000     9.0000   -0.4836
goal             5000.0000 6214.9004 5021.0111 1000.0000 2500.0000 5000.0000 9000.0000 24800.0000    1.3300
images           5000.0000    4.7046    7.9015    0.0000    0.0000    2.0000    6.0000   109.0000    4.2722
video            5000.0000    0.5852    0.4927    0.0000    0.0000    1.0000    1.0000     1.0000   -0.3460
comments_count   5000.0000   16.7560   26.3605    0.0000    5.0000   10.0000   20.0000   745.0000   11.6172
updates_count    5000.0000    4.0142    7.4101    0.0000    0.0000    1.0000    5.0000   169.0000    5.9076
creators         5000.0000    2.1634    2.0532    1.0000    1.0000    1.0000    3.0000    32.0000    3.9795
bus

In [6]:
# CELL 6: CORRELATION ANALYSIS
print("\n" + "="*70)
print("CORRELATION ANALYSIS")
print("="*70)

df_camp['reach30in2_num'] = df_camp['reach30in2'].astype(int)

corr_vars = ['collected_funds', 'goal', 'campaign_quality', 'images', 'video',
             'pitch_size', 'comments_count', 'updates_count', 'creators',
             'business_venture', 'social', 'filler_words', 'posemo', 'negemo',
             'reach30in2_num', 'duration', 'time']

corr_matrix = df_camp[corr_vars].corr()

print("\nCorrelations with collected_funds:")
print(corr_matrix['collected_funds'].sort_values(ascending=False).to_string())

# heatmap
plt.figure(figsize=(14, 10))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f',
            cmap='RdBu_r', center=0, square=True, linewidths=0.5,
            annot_kws={'size': 8})
plt.title('Correlation Matrix')
plt.tight_layout()
plt.show()

print("\nKey finding: comments_count has strongest correlation (r=0.68)")


CORRELATION ANALYSIS

Correlations with collected_funds:
collected_funds     1.0000
comments_count      0.6757
goal                0.3110
updates_count       0.2274
images              0.1347
pitch_size          0.1266
campaign_quality    0.1258
creators            0.1039
video               0.0762
reach30in2_num      0.0529
time                0.0476
negemo              0.0336
duration            0.0239
social              0.0130
filler_words       -0.0039
posemo             -0.0200
business_venture   -0.0239

Key finding: comments_count has strongest correlation (r=0.68)


In [7]:
# CELL 7: HYPOTHESIS 1 - CAMPAIGN QUALITY
print("\n" + "="*70)
print("HYPOTHESIS 1: CAMPAIGN QUALITY & SIGNALING")
print("="*70)
print("""
H1: Campaigns with higher quality signals (images, videos, detailed pitch)
    will collect more funds.

Reasoning: Quality content signals entrepreneur commitment and reduces
perceived risk for backers (signaling theory).
""")

y = df_camp['collected_funds']
X_h1 = df_camp[['goal', 'campaign_quality', 'images', 'video', 'pitch_size',
                'time', 'duration']]
X_h1 = sm.add_constant(X_h1)

# extra/supplementary - HC3 robust SE is a specific implementation choice
model_h1 = sm.OLS(y, X_h1).fit(cov_type='HC3')
print("\nModel 1: Quality Variables")
print(model_h1.summary())

print("\nH1 Results:")
for var in ['campaign_quality', 'images', 'video', 'pitch_size']:
    coef = model_h1.params[var]
    pval = model_h1.pvalues[var]
    sig = "***" if pval < 0.001 else "**" if pval < 0.01 else "*" if pval < 0.05 else ""
    print(f"   {var}: coef = {coef:.2f}, p = {pval:.4f} {sig}")


HYPOTHESIS 1: CAMPAIGN QUALITY & SIGNALING

H1: Campaigns with higher quality signals (images, videos, detailed pitch)
    will collect more funds.
    
Reasoning: Quality content signals entrepreneur commitment and reduces
perceived risk for backers (signaling theory).


Model 1: Quality Variables
                            OLS Regression Results                            
Dep. Variable:        collected_funds   R-squared:                       0.113
Model:                            OLS   Adj. R-squared:                  0.112
Method:                 Least Squares   F-statistic:                     47.70
Date:                Tue, 06 Jan 2026   Prob (F-statistic):           6.12e-66
Time:                        23:00:25   Log-Likelihood:                -48157.
No. Observations:                5000   AIC:                         9.633e+04
Df Residuals:                    4992   BIC:                         9.638e+04
Df Model:                           7                              

In [8]:
# CELL 8: HYPOTHESIS 2 - ENGAGEMENT & SOCIAL PROOF
print("\n" + "="*70)
print("HYPOTHESIS 2: ENGAGEMENT & SOCIAL PROOF")
print("="*70)
print("""
H2: Campaigns with more backer engagement (comments) and creator updates
    will collect more funds.

Reasoning: Visible interest from others signals quality (social proof/herding).
Early momentum (reach30in2) creates positive feedback loops.

Note: comments_count measured at end, so potential reverse causality.
""")

X_h2 = df_camp[['goal', 'comments_count', 'updates_count', 'reach30in2_num',
                'time', 'duration']]
X_h2 = sm.add_constant(X_h2)

# extra/supplementary - HC3 robust SE
model_h2 = sm.OLS(y, X_h2).fit(cov_type='HC3')
print("\nModel 2: Engagement Variables")
print(model_h2.summary())

print("\nH2 Results:")
for var in ['comments_count', 'updates_count', 'reach30in2_num']:
    coef = model_h2.params[var]
    pval = model_h2.pvalues[var]
    sig = "***" if pval < 0.001 else "**" if pval < 0.01 else "*" if pval < 0.05 else ""
    print(f"   {var}: coef = {coef:.2f}, p = {pval:.4f} {sig}")


HYPOTHESIS 2: ENGAGEMENT & SOCIAL PROOF

H2: Campaigns with more backer engagement (comments) and creator updates
    will collect more funds.
    
Reasoning: Visible interest from others signals quality (social proof/herding).
Early momentum (reach30in2) creates positive feedback loops.

Note: comments_count measured at end, so potential reverse causality.


Model 2: Engagement Variables
                            OLS Regression Results                            
Dep. Variable:        collected_funds   R-squared:                       0.517
Model:                            OLS   Adj. R-squared:                  0.516
Method:                 Least Squares   F-statistic:                     90.60
Date:                Tue, 06 Jan 2026   Prob (F-statistic):          2.72e-108
Time:                        23:00:44   Log-Likelihood:                -46640.
No. Observations:                5000   AIC:                         9.329e+04
Df Residuals:                    4993   BIC:          

In [9]:
# CELL 9: HYPOTHESIS 3 - PROJECT TYPE
print("\n" + "="*70)
print("HYPOTHESIS 3: PROJECT TYPE & PROFESSIONALISM")
print("="*70)
print("""
H3: Business ventures and social causes will collect more funds than
    personal projects. Filler words indicate less professionalism.

Reasoning: Business/social projects signal seriousness and accountability.
""")

X_h3 = df_camp[['goal', 'business_venture', 'social', 'filler_words',
                'creators', 'time', 'duration']]
X_h3 = sm.add_constant(X_h3)

# extra/supplementary - HC3 robust SE
model_h3 = sm.OLS(y, X_h3).fit(cov_type='HC3')
print("\nModel 3: Project Type Variables")
print(model_h3.summary())

print("\nH3 Results:")
for var in ['business_venture', 'social', 'filler_words', 'creators']:
    coef = model_h3.params[var]
    pval = model_h3.pvalues[var]
    sig = "***" if pval < 0.001 else "**" if pval < 0.01 else "*" if pval < 0.05 else ""
    print(f"   {var}: coef = {coef:.2f}, p = {pval:.4f} {sig}")


HYPOTHESIS 3: PROJECT TYPE & PROFESSIONALISM

H3: Business ventures and social causes will collect more funds than
    personal projects. Filler words indicate less professionalism.
    
Reasoning: Business/social projects signal seriousness and accountability.


Model 3: Project Type Variables
                            OLS Regression Results                            
Dep. Variable:        collected_funds   R-squared:                       0.106
Model:                            OLS   Adj. R-squared:                  0.104
Method:                 Least Squares   F-statistic:                     46.34
Date:                Tue, 06 Jan 2026   Prob (F-statistic):           5.06e-64
Time:                        23:00:58   Log-Likelihood:                -48178.
No. Observations:                5000   AIC:                         9.637e+04
Df Residuals:                    4992   BIC:                         9.642e+04
Df Model:                           7                                  

In [10]:
# CELL 10: FULL MODEL
print("\n" + "="*70)
print("FULL MODEL: ALL HYPOTHESES COMBINED")
print("="*70)

full_vars = ['goal',
             'campaign_quality', 'images', 'video', 'pitch_size',  # H1
             'comments_count', 'updates_count', 'reach30in2_num',   # H2
             'business_venture', 'social', 'filler_words', 'creators',  # H3
             'time', 'duration']  # controls

X_full = df_camp[full_vars]
X_full = sm.add_constant(X_full)

# extra/supplementary - HC3 robust SE
model_full = sm.OLS(y, X_full).fit(cov_type='HC3')
print("\nFull Model (OLS with robust SE)")
print(model_full.summary())

print(f"\nModel Fit:")
print(f"   R-squared: {model_full.rsquared:.4f}")
print(f"   Adj R-squared: {model_full.rsquared_adj:.4f}")


FULL MODEL: ALL HYPOTHESES COMBINED

Full Model (OLS with robust SE)
                            OLS Regression Results                            
Dep. Variable:        collected_funds   R-squared:                       0.518
Model:                            OLS   Adj. R-squared:                  0.516
Method:                 Least Squares   F-statistic:                     46.63
Date:                Tue, 06 Jan 2026   Prob (F-statistic):          4.91e-122
Time:                        23:01:09   Log-Likelihood:                -46635.
No. Observations:                5000   AIC:                         9.330e+04
Df Residuals:                    4985   BIC:                         9.340e+04
Df Model:                          14                                         
Covariance Type:                  HC3                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------

In [11]:
# CELL 11: PANEL DATA ANALYSIS
# extra/supplementary - Random Effects panel model via linearmodels not required by course
print("\n" + "="*70)
print("PANEL DATA ANALYSIS (SUPPLEMENTARY)")
print("="*70)
print("""
Using contribution dataset to analyze daily contributions.
This helps address endogeneity since predictors are measured before contributions.
""")

try:
    from linearmodels.panel import RandomEffects

    df_panel = df_cont.set_index(['id', 'day'])
    panel_vars = ['pitch_size', 'posemo', 'negemo', 'focuspast',
                  'focuspresent', 'focusfuture', 'creators']

    df_panel_clean = df_panel.dropna(subset=panel_vars + ['dailycontrib'])
    y_panel = df_panel_clean['dailycontrib']
    X_panel = df_panel_clean[panel_vars]
    X_panel = sm.add_constant(X_panel)

    re_model = RandomEffects(y_panel, X_panel).fit()
    print("\nRandom Effects Model:")
    print(re_model)

except ImportError:
    print("\nlinearmodels not installed, running pooled OLS instead...")
    X_pooled = df_cont[['pitch_size', 'posemo', 'negemo', 'focuspast',
                        'focuspresent', 'focusfuture', 'creators']]
    X_pooled = sm.add_constant(X_pooled)
    y_pooled = df_cont['dailycontrib']
    pooled_model = sm.OLS(y_pooled, X_pooled).fit(cov_type='cluster',
                          cov_kwds={'groups': df_cont['id']})
    print(pooled_model.summary())


PANEL DATA ANALYSIS (SUPPLEMENTARY)

Using contribution dataset to analyze daily contributions.
This helps address endogeneity since predictors are measured before contributions.


linearmodels not installed, running pooled OLS instead...
                            OLS Regression Results                            
Dep. Variable:           dailycontrib   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.201
Date:                Tue, 06 Jan 2026   Prob (F-statistic):              0.304
Time:                        23:01:21   Log-Likelihood:                -15112.
No. Observations:                8000   AIC:                         3.024e+04
Df Residuals:                    7992   BIC:                         3.029e+04
Df Model:                           7                                         
Covariance Type:              cluster            

In [12]:
# CELL 12: MODEL DIAGNOSTICS
# extra/supplementary - formal diagnostic tests (JB, BP, VIF) not required by course
print("\n" + "="*70)
print("MODEL DIAGNOSTICS (SUPPLEMENTARY)")
print("="*70)

resid = model_full.resid

# Jarque-Bera normality test
print("\n1. Normality of Residuals (Jarque-Bera):")
jb_stat, jb_pval, skew, kurtosis = sms.jarque_bera(resid)
print(f"   JB Stat: {jb_stat:.2f}, p-value: {jb_pval:.4g}")
print(f"   Skewness: {skew:.2f}, Kurtosis: {kurtosis:.2f}")
if jb_pval < 0.05:
    print("   Residuals not normally distributed (common with large N)")

# Breusch-Pagan heteroskedasticity test
print("\n2. Heteroskedasticity (Breusch-Pagan):")
bp_stat, bp_pval, f_stat, f_pval = sms.het_breuschpagan(resid, model_full.model.exog)
print(f"   LM Stat: {bp_stat:.2f}, p-value: {bp_pval:.4g}")
if bp_pval < 0.05:
    print("   Heteroskedasticity detected - using robust SE addresses this")

# VIF for multicollinearity
print("\n3. Multicollinearity (VIF):")
exog_df = pd.DataFrame(model_full.model.exog, columns=model_full.model.exog_names)
for i, col in enumerate(exog_df.columns):
    if col == 'const':
        continue
    vif = variance_inflation_factor(exog_df.values, i)
    flag = " (HIGH)" if vif > 10 else ""
    print(f"   {col}: {vif:.2f}{flag}")

# basic fit stats
print("\n4. Model Fit:")
print(f"   R-squared: {model_full.rsquared:.4f}")
print(f"   Adj R-squared: {model_full.rsquared_adj:.4f}")
print(f"   F-stat: {model_full.fvalue:.2f}, p = {model_full.f_pvalue:.4g}")
print(f"   RMSE: {np.sqrt(np.mean(resid**2)):.2f}")


MODEL DIAGNOSTICS (SUPPLEMENTARY)

1. Normality of Residuals (Jarque-Bera):
   JB Stat: 259877.09, p-value: 0
   Skewness: 2.53, Kurtosis: 37.95
   Residuals not normally distributed (common with large N)

2. Heteroskedasticity (Breusch-Pagan):
   LM Stat: 1046.31, p-value: 1.808e-214
   Heteroskedasticity detected - using robust SE addresses this

3. Multicollinearity (VIF):
   goal: 1.15
   campaign_quality: 3.39
   images: 1.84
   video: 2.59
   pitch_size: 1.12
   comments_count: 1.11
   updates_count: 1.67
   reach30in2_num: 1.06
   business_venture: 3.41
   social: 3.48
   filler_words: 1.00
   creators: 1.08
   time: 1.17
   duration: 1.13

4. Model Fit:
   R-squared: 0.5176
   Adj R-squared: 0.5162
   F-stat: 46.63, p = 4.906e-122
   RMSE: 2718.93


In [13]:
# CELL 13: VISUALIZATIONS
print("\n" + "="*70)
print("VISUALIZATIONS")
print("="*70)

fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# distribution of collected funds
ax1 = axes[0, 0]
ax1.hist(df_camp['collected_funds'], bins=50, edgecolor='white', alpha=0.7)
ax1.axvline(df_camp['collected_funds'].mean(), color='red', linestyle='--',
            label=f'Mean: ${df_camp["collected_funds"].mean():,.0f}')
ax1.axvline(df_camp['collected_funds'].median(), color='orange', linestyle='--',
            label=f'Median: ${df_camp["collected_funds"].median():,.0f}')
ax1.set_xlabel('Collected Funds ($)')
ax1.set_ylabel('Frequency')
ax1.set_title('Distribution of Collected Funds')
ax1.legend()

# scatter: comments vs funds
ax2 = axes[0, 1]
ax2.scatter(df_camp['comments_count'], df_camp['collected_funds'], alpha=0.3, s=10)
z = np.polyfit(df_camp['comments_count'], df_camp['collected_funds'], 1)
p = np.poly1d(z)
ax2.plot(sorted(df_camp['comments_count']), p(sorted(df_camp['comments_count'])),
         "r--", linewidth=2)
ax2.set_xlabel('Comments Count')
ax2.set_ylabel('Collected Funds ($)')
ax2.set_title('Comments vs Funds (r=0.68)')

# boxplot by business venture
ax3 = axes[0, 2]
df_camp.boxplot(column='collected_funds', by='business_venture', ax=ax3)
ax3.set_xlabel('Business Venture (0=No, 1=Yes)')
ax3.set_ylabel('Collected Funds ($)')
ax3.set_title('Funds by Business Venture')
plt.suptitle('')

# log distribution
ax4 = axes[1, 0]
ax4.hist(np.log(df_camp['collected_funds']), bins=50, edgecolor='white', alpha=0.7)
ax4.set_xlabel('Log(Collected Funds)')
ax4.set_ylabel('Frequency')
ax4.set_title('Log-Transformed Distribution')

# residuals vs fitted
ax5 = axes[1, 1]
ax5.scatter(model_full.fittedvalues, resid, alpha=0.3, s=10)
ax5.axhline(y=0, color='red', linestyle='--')
ax5.set_xlabel('Fitted Values')
ax5.set_ylabel('Residuals')
ax5.set_title('Residuals vs Fitted')

# QQ plot
ax6 = axes[1, 2]
stats.probplot(resid, dist="norm", plot=ax6)
ax6.set_title('Q-Q Plot')

plt.tight_layout()
plt.savefig('crowdfunding_plots.png', dpi=150)
plt.show()
print("Plots saved")


VISUALIZATIONS
Plots saved


In [14]:
# CELL 14: ROBUSTNESS CHECKS
# extra/supplementary - multiple robustness variants not required by course
print("\n" + "="*70)
print("ROBUSTNESS CHECKS (SUPPLEMENTARY)")
print("="*70)

# log-log specification
print("\n1. Log-Log Specification:")
df_log = df_camp.copy()
df_log['ln_funds'] = np.log(df_log['collected_funds'])
df_log['ln_goal'] = np.log(df_log['goal'])
df_log['ln_images'] = np.log(df_log['images'] + 1)
df_log['ln_pitch'] = np.log(df_log['pitch_size'])
df_log['ln_comments'] = np.log(df_log['comments_count'] + 1)
df_log['ln_updates'] = np.log(df_log['updates_count'] + 1)
df_log['ln_quality'] = np.log(df_log['campaign_quality'] + 1)

log_vars = ['ln_goal', 'ln_quality', 'ln_images', 'video', 'ln_pitch',
            'ln_comments', 'ln_updates', 'reach30in2_num',
            'business_venture', 'social', 'filler_words', 'creators',
            'time', 'duration']

X_log = df_log[log_vars]
X_log = sm.add_constant(X_log)
y_log = df_log['ln_funds']

model_log = sm.OLS(y_log, X_log).fit(cov_type='HC3')
print(model_log.summary())
print(f"\n   Log-Log R²: {model_log.rsquared:.4f}")
print(f"   Level R²: {model_full.rsquared:.4f}")

# funding ratio as alternative DV
print("\n2. Alternative DV (Funding Ratio):")
y_ratio = df_camp['funding_ratio']
model_ratio = sm.OLS(y_ratio, X_full).fit(cov_type='HC3')

print("\n   Key coefficients comparison:")
print("   Variable            Levels      Ratio")
for var in ['comments_count', 'campaign_quality', 'images', 'updates_count']:
    c1 = model_full.params[var]
    c2 = model_ratio.params[var]
    p1 = "*" if model_full.pvalues[var] < 0.05 else ""
    p2 = "*" if model_ratio.pvalues[var] < 0.05 else ""
    print(f"   {var:20} {c1:8.4f}{p1}   {c2:.6f}{p2}")

# model comparison
print("\n3. Model Comparison:")
print(f"   H1 Only:     R² = {model_h1.rsquared:.4f}")
print(f"   H2 Only:     R² = {model_h2.rsquared:.4f}")
print(f"   H3 Only:     R² = {model_h3.rsquared:.4f}")
print(f"   Full Model:  R² = {model_full.rsquared:.4f}")
print(f"   Log-Log:     R² = {model_log.rsquared:.4f}")


ROBUSTNESS CHECKS (SUPPLEMENTARY)

1. Log-Log Specification:
                            OLS Regression Results                            
Dep. Variable:               ln_funds   R-squared:                       0.400
Model:                            OLS   Adj. R-squared:                  0.398
Method:                 Least Squares   F-statistic:                     212.3
Date:                Tue, 06 Jan 2026   Prob (F-statistic):               0.00
Time:                        23:02:20   Log-Likelihood:                -5078.4
No. Observations:                5000   AIC:                         1.019e+04
Df Residuals:                    4985   BIC:                         1.028e+04
Df Model:                          14                                         
Covariance Type:                  HC3                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------

# CELL 15: CONCLUSIONS

In [15]:
print("\n" + "="*70)
print("CONCLUSIONS")
print("="*70)

print("""
RESEARCH QUESTION: What factors lead to successful crowdfunding campaigns?
""")

# H1 results
print("HYPOTHESIS 1: CAMPAIGN QUALITY")
print("-" * 40)
h1_vars = ['campaign_quality', 'images', 'video', 'pitch_size']
h1_support = 0
for var in h1_vars:
    coef = model_full.params[var]
    pval = model_full.pvalues[var]
    sig = "SUPPORTED" if (pval < 0.05 and coef > 0) else "Not significant"
    if pval < 0.05 and coef > 0:
        h1_support += 1
    print(f"   {var}: coef={coef:.2f}, p={pval:.4f} -> {sig}")
print(f"\n   H1 VERDICT: {'PARTIALLY SUPPORTED' if h1_support > 0 else 'NOT SUPPORTED'}")

# H2 results
print("\nHYPOTHESIS 2: ENGAGEMENT & SOCIAL PROOF")
print("-" * 40)
h2_vars = ['comments_count', 'updates_count', 'reach30in2_num']
h2_support = 0
for var in h2_vars:
    coef = model_full.params[var]
    pval = model_full.pvalues[var]
    sig = "SUPPORTED" if (pval < 0.05 and coef > 0) else "Not significant"
    if pval < 0.05 and coef > 0:
        h2_support += 1
    print(f"   {var}: coef={coef:.2f}, p={pval:.4f} -> {sig}")
print(f"\n   H2 VERDICT: {'STRONGLY SUPPORTED' if h2_support >= 2 else 'PARTIALLY SUPPORTED' if h2_support > 0 else 'NOT SUPPORTED'}")

# H3 results
print("\nHYPOTHESIS 3: PROJECT TYPE")
print("-" * 40)
h3_vars = ['business_venture', 'social', 'filler_words', 'creators']
h3_support = 0
for var in h3_vars:
    coef = model_full.params[var]
    pval = model_full.pvalues[var]
    if var == 'filler_words':
        sig = "SUPPORTED" if (pval < 0.05 and coef < 0) else "Not significant"
        if pval < 0.05 and coef < 0:
            h3_support += 1
    else:
        sig = "SUPPORTED" if (pval < 0.05 and coef > 0) else "Not significant"
        if pval < 0.05 and coef > 0:
            h3_support += 1
    print(f"   {var}: coef={coef:.2f}, p={pval:.4f} -> {sig}")
print(f"\n   H3 VERDICT: {'PARTIALLY SUPPORTED' if h3_support > 0 else 'NOT SUPPORTED'}")

# overall summary
print("\n" + "="*70)
print("SUMMARY")
print("="*70)
print(f"""
Key Findings:
- Engagement (comments) is the strongest predictor of success (r=0.68)
- Goal size matters - larger campaigns raise more in absolute terms
- Updates and early momentum (reach30in2) also matter

What matters less than expected:
- Campaign quality signals (images, video) have weak effects in full model
- Emotional language (posemo/negemo) not significant
- Business/social categories only marginally significant

Model Performance:
- Full model R² = {model_full.rsquared:.4f} (explains {model_full.rsquared*100:.1f}% of variance)
- H2 variables contribute most to explanatory power

Limitations:
- Potential endogeneity in comments (reverse causality)
- Cross-sectional data limits causal claims
- Sample not fully representative
""")

print("="*70)
print("END OF ANALYSIS")
print("="*70)


CONCLUSIONS

RESEARCH QUESTION: What factors lead to successful crowdfunding campaigns?

HYPOTHESIS 1: CAMPAIGN QUALITY
----------------------------------------
   campaign_quality: coef=29.03, p=0.3443 -> Not significant
   images: coef=-10.82, p=0.1696 -> Not significant
   video: coef=-104.57, p=0.3915 -> Not significant
   pitch_size: coef=0.02, p=0.8832 -> Not significant

   H1 VERDICT: NOT SUPPORTED

HYPOTHESIS 2: ENGAGEMENT & SOCIAL PROOF
----------------------------------------
   comments_count: coef=94.10, p=0.0000 -> SUPPORTED
   updates_count: coef=34.17, p=0.0179 -> SUPPORTED
   reach30in2_num: coef=329.51, p=0.0108 -> SUPPORTED

   H2 VERDICT: STRONGLY SUPPORTED

HYPOTHESIS 3: PROJECT TYPE
----------------------------------------
   business_venture: coef=359.75, p=0.0656 -> Not significant
   social: coef=402.23, p=0.0580 -> Not significant
   filler_words: coef=-301.16, p=0.7464 -> Not significant
   creators: coef=22.54, p=0.3662 -> Not significant

   H3 VERDICT: NO